# Experimental Design
***
## Magnetism

We can also make use of magnetic layers in HOGBEN. The syntax is similar to the non-magnetic layers, and spin-states should be handled automatically. At the moment, the magnetic layers are completely defined in terms of their magnetic SLD.

Import the necessary packages and code to use magnetic layers.

In [ ]:
%matplotlib inline

from refnx.reflect import SLD
from refnx.analysis import Parameter
from hogben.models.samples import Sample
from hogben.models.base import MagneticSLD
from hogben.optimise import optimise_parameters

We can create a magnetic layer using the `MagneticSLD` module in HOGBEN. The syntax mostly works the same as ordinary layers in `refnx`, with the difference that the magnetic part of the SLD should be given as a second argument in `MagneticSLD`. If no magnetic SLD is provided, the magnetic SLD defaults to 0. In this example, we will create a simple sample with a magnetic reference layer (MRL).

In [ ]:
def simple_sample():
    """Define a bilayer sample, and return the associated refnx model"""
    
    # Define the fitting parameters for the sample:
    layer1_thick = Parameter(80, 'Layer 1 Thickness', (50, 120))
    layer2_thick = Parameter(40, 'Layer 2 Thickness', (30, 50))
    layer3_thick = Parameter(60, 'Layer 3 Thickness', (50, 120))    
    
    # Define the parameters for the reference layer that we want to optimize
    ref_thick = Parameter(50, 'Reference layer Thickness', (0, 400))
    ref_sld = Parameter(3, 'Reference layer SLD', (-1.9, 9.4))
    ref_msld = Parameter(3, 'Reference layer Magnetic SLD', (0, 7))
    
    # Construct the layers
    air = SLD(0, name='Air')
    layer1 = SLD(6.5, name="Layer 1")(thick=layer1_thick, rough=0)
    layer2 = SLD(1.5, name="Layer 2")(thick=layer2_thick, rough=0)
    layer3 = SLD(4.5, name="Layer 3")(thick=layer3_thick, rough=0)   
    ref_layer_m = MagneticSLD(ref_sld, ref_msld, name="Layer 3")(thick=ref_thick, rough=0)
    substrate = SLD(2.074, name='Substrate')(rough=0)

    # Create a structure, separating each layer with a `|`
    structure = air | layer3 | layer2 | layer1 | ref_layer_m | substrate
    return structure

We can now create our sample using the `Sample` module, and the spin-states are handled automatically. 

In [ ]:
sample_structure = simple_sample()
magnetic_sample = Sample(sample_structure)
magnetic_sample.reflectivity_profile()
magnetic_sample.sld_profile()

When initialising the sample, we can choose to turn the polarisation state on or off for the experiment. When the polarisation state is set to `False`, then the magnetic part of the SLD is not used at all. For example:

In [ ]:
magnetic_sample = Sample(sample_structure, polarised=False)
magnetic_sample.reflectivity_profile()
magnetic_sample.sld_profile()

## Setting instrument settings

We can also set the instrument settings for the sample. We can set the background, resolution and scale using `bkg`, `dq` and `scale` respectively. See the example below: 

In [ ]:
sample_structure = simple_sample()
magnetic_sample = Sample(sample_structure, bkg=5e-6, dq=2, scale=1.2, polarised=False)
magnetic_sample.reflectivity_profile()
magnetic_sample.sld_profile()

When no value is provided, a default value of `5e-6`, `2` and `1` will be used for the background, resolution and scaling respectively.

### Setting instrument settings with multiple structures

If we use multiple structures, for example when measuring the same sample in two different ambients, then we can use a different setting for each structure by providing these as a list. See the example below, where we define a sample with two structures:

In [ ]:
def simple_sample_two_structures():
    """Define a bilayer sample, and return the associated refnx model"""
    
    # Define the fitting parameters for the sample:
    layer1_thick = Parameter(80, 'Layer 1 Thickness', (50, 120))
    layer2_thick = Parameter(40, 'Layer 2 Thickness', (30, 50))
    layer3_thick = Parameter(60, 'Layer 3 Thickness', (50, 120))    
    
    # Define the parameters for the reference layer that we want to optimize
    ref_thick = Parameter(50, 'Reference layer Thickness', (0, 400))
    ref_sld = Parameter(3, 'Reference layer SLD', (-1.9, 9.4))
    ref_msld = Parameter(3, 'Reference layer Magnetic SLD', (0, 7))
    
    # Construct the layers
    H2O = SLD(-0.52, name='H2O')
    D2O = SLD(6.19, name='D2O')    
    layer1 = SLD(6.5, name="Layer 1")(thick=layer1_thick, rough=0)
    layer2 = SLD(1.5, name="Layer 2")(thick=layer2_thick, rough=0)
    layer3 = SLD(4.5, name="Layer 3")(thick=layer3_thick, rough=0)   
    ref_layer_m = MagneticSLD(ref_sld, ref_msld, name="Layer 3")(thick=ref_thick, rough=0)
    substrate = SLD(2.074, name='Substrate')(rough=0)

    # Create a structure, separating each layer with a `|`
    structure_H2O = H2O | layer3 | layer2 | layer1 | ref_layer_m | substrate
    structure_D2O = D2O | layer3 | layer2 | layer1 | ref_layer_m | substrate
    
    return [structure_H2O, structure_D2O]

The sample can be initiated as usual. If we want to use different settings for each structure, for instance using different backgrounds when measured in H2O as to measured in D2O, then the settings can be given as a list in the same order as the order in which the structures were given. Alternatively, a single value can be given to use the same value for each structure.

Additionally, a list of labels can be given for each structure to be used in the legend using the `labels` attribute, if the labels are not set a label will be generated automatically.

In [ ]:
sample_structure = simple_sample_two_structures()
magnetic_sample = Sample(sample_structure, bkg=[2e-6, 4e-6], dq=2, scale=1.2, labels=['H2O Solvent', 'D2O Solvent'])
magnetic_sample.reflectivity_profile()
magnetic_sample.sld_profile()

## Fisher optimization of magnetic multilayers

Working with magnetic layers works exactly the same as non-magnetic layers. Magnetic parameters can thus also be optimized. 
For these optimizations, the experiment is simulated twice with equal measuring time in the spin-up as well as in the spin-down direction. As a simple approximation, the neutron flux is halved for polarised experiments, since most polarisation methods lose at least half the neutrons. We also make the (often untrue) assumption that the sample is under-illuminated, so the flux would need to be scaled appropriately for accurate values. All ratios of times will be accurate however.

In this example, we have a material consisting of three layers and a magnetic reference layer (MRL). Here we try to predict the ideal MRL in order to get the most information about the roughness and thickness of the three layers on top.  

In [ ]:
def simple_sample_MRL():
    """Define a bilayer sample, and return the associated refnx model"""
    
    # Define the fitting parameters for the sample:
    layer1_thick = Parameter(80, 'Layer 1 Thickness', (50, 120))
    layer2_thick = Parameter(40, 'Layer 2 Thickness', (30, 50))
    layer3_thick = Parameter(60, 'Layer 3 Thickness', (50, 120))    
    layer1_rough = Parameter(4, 'Layer 1 Roughness', (2, 10))
    layer2_rough = Parameter(5, 'Layer 2 Roughness', (2, 10))
    layer3_rough = Parameter(3, 'Layer 3 Roughness', (50, 120))
    
    # Define the parameters for the reference layer that we want to optimize
    ref_thick = Parameter(0, 'Reference layer Thickness', (0, 200))
    ref_sld = Parameter(3, 'Reference layer Nuclear SLD', (0, 9.4))
    ref_msld = Parameter(3, 'Reference layer Magnetic SLD', (0, 7))

    # Tell HOGBEN that these parameters should be optimized
    ref_thick.optimize = True
    ref_sld.optimize = True
    ref_msld.optimize = True
    
    # Construct the layers
    air = SLD(0, name='Air')
    layer1 = SLD(6.5, name="Layer 1")(thick=layer1_thick, rough=layer1_rough)
    layer2 = SLD(1.5, name="Layer 2")(thick=layer2_thick, rough=layer2_rough)
    layer3 = SLD(4.5, name="Layer 3")(thick=layer3_thick, rough=layer3_rough)   
    ref_layer_m = MagneticSLD(ref_sld, ref_msld, name = "Reference layer")(thick=ref_thick, rough=0)

    substrate = SLD(2.074, name='Substrate')(rough=0)

    # Put all fitting parameters in a list
    params = [
        layer1_rough,
        layer2_rough,
        layer3_rough,
        layer1_thick,
        layer2_thick,
        layer3_thick,
    ]
    
    # Set all fitting parameters to be varying
    for param in params:
        param.vary = True
    
    # Create a structure, separating each layer with a `|`
    structure = air | layer3 | layer2 | layer1 | ref_layer_m | substrate
    return structure

Now we can simply create a sample using the defined structure, and perform Fisher optimization using `optimise_parameters`.

In [ ]:
sample_structure = simple_sample_MRL()
magnetic_sample = Sample(sample_structure, polarised=True)
angle_times = [(0.4, 100, 100),
               (2.3, 100, 400),
               ]    
optimise_parameters(magnetic_sample, angle_times)

We can do the same optimization as well for an unpolarised experiment, this can be simply done by initialising the sample with `polarised=False`, or alternatively setting `magnetic_sample.polarised = False` after it has been initialized. 

In [ ]:
sample_structure = simple_sample_MRL()
magnetic_sample = Sample(sample_structure, polarised=False)
angle_times = [(0.4, 100, 100),
               (2.3, 100, 400),
               ]
optimise_parameters(magnetic_sample, angle_times)